# Functions for coordinate conversion

* Observer is OVRO 40m telescope 
* Assumes all the coordinates are J2000.

Basic functions for coordinate/time transformations and source properties.

**pip install ephemWalter Max-Moerbeck, 26 January 2009**

In [1]:
#pip install ephem
#pip install ipynb

In [2]:
#import telescope_data as tel_data
import ipynb.fs.full.telescope_data as tel_data
import numpy
import math
import ephem

In [3]:
# Conversion factors
rad_to_deg = 180. / math.pi
deg_to_rad = math.pi / 180.

In [4]:
def radec_zaaz(ra, dec, lst):
    """ Zenith/Azimuth for a source given RA/DEC and observer position.

    INPUT:
    ra:        RA decimal degrees
    dec:       DEC decimal degrees
    lst:       Decimal hours
    
    OUTPUT:
    za, az:    ZA/AZ coordinates decimal degrees


    This simple recipe doesn't consider observer elevation nor refraction 
    effects, etc. It should be off by some arcminutes, this is probably not a 
    problem for the optimizer/scheduler. Of course it is not good enough to 
    point a telescope!
    """
    
    # Wrap around to [0, 24) range
    lst = lst % 24

    # Observer coordinates
    long = tel_data.lon
    lat = tel_data.lat
    
    # Local Hour Angle.
    # 15 on ra is to convert to decimal hours
    # 15 on total converts from hours to degrees
    lha = (lst - ra / 15.0) * 15.0

    # convert everything to radians
    long = long * math.pi / 180.0
    lat = lat * math.pi / 180.0
    lha = lha * math.pi / 180.0
    dec = dec * math.pi / 180.0

    # za in radians
    za = math.pi / 2.0 - \
        math.asin(math.cos(lha) * math.cos(dec) * math.cos(lat) + \
                      math.sin(dec) * math.sin(lat))

    # az in radians. -180 < atan2 < 180. But 0 < az < 360
    az = math.atan2( -math.sin(lha) * math.cos(dec),\
                          -math.sin(lat) * math.cos(lha) * math.cos(dec) + \
                          math.sin(dec) * math.cos(lat) )

    # to degrees
    za = za * 180.0 / math.pi
    az = az * 180.0 / math.pi

    # 0 < az < 360
    if az < 0:
        az = az + 360.0

    return za, az

In [5]:
def dis_zaaz(za1, az1, za2, az2):
    """ Angular distance in ZAAZ coordinates.

    INPUT:
    za, az:    Source coordinates in decimal degrees 

    OUTPUT:
    dis:       Angular distance in degrees
    """

    # use dot product of two unitary vectors

    # convert to spherical coordinates and radians
    theta1 = za1 * math.pi / 180.
    phi1 = (360. - az1) * math.pi / 180.
    theta2 = za2 * math.pi / 180.
    phi2 = (360. - az2) * math.pi / 180.

    # calculate vectors
    r1 = numpy.array([math.sin(theta1) * math.cos(phi1),\
                    math.sin(theta1) * math.sin(phi1),\
                    math.cos(theta1)])

    r2 = numpy.array([math.sin(theta2) * math.cos(phi2),\
                    math.sin(theta2) * math.sin(phi2),\
                    math.cos(theta2)])
    
    # get angle between vectors in degrees
    dis = math.acos( min(numpy.sum(r1 * r2), 1) ) * 180. / math.pi

    return dis

In [6]:
def dis_radec(ra1, dec1, ra2, dec2):
    """ Angular distance in RADEC coordinates.
    ra, dec and final distance are in degrees
    """    

    # convert to radians
    ra1 = ra1 * math.pi / 180.0
    dec1 = dec1 * math.pi / 180.0
    ra2 = ra2 * math.pi / 180.0
    dec2 = dec2 * math.pi / 180.0

    # calculate the vectors
    r1 = numpy.array([math.cos(dec1) * math.cos(ra1),\
                    math.cos(dec1) * math.sin(ra1),\
                    math.sin(dec1)])

    r2 = numpy.array([math.cos(dec2) * math.cos(ra2),\
                    math.cos(dec2) * math.sin(ra2),\
                    math.sin(dec2)])


    # get angle between vectors in degrees
    dis = math.acos( min(numpy.sum(r1 * r2), 1) ) * 180. / math.pi
    
    return dis

In [7]:
def jd_to_djd(jd):
    """ Convert from JD to DJD
    """

    djd = jd - 2415020.0

    return djd

In [8]:
def sun_pos(jd):
    """ Sun position in ZAAZ coordinates given JD.
    
    Uses PyEphem to get position.
    """
        
    # convert to Dublin Julian Day as used by PyEphem
    djd = jd_to_djd(jd)

    # get postion of Sun
    Sun = ephem.Sun(djd)

    # Angles in decimal degrees for ra and dec
    ra = Sun.ra * rad_to_deg
    dec = Sun.dec * rad_to_deg

    return ra, dec

In [9]:
def moon_pos(jd):
    """ Moon position in ZAAZ coordinates given JD.
    
    Uses PyEphem ro get position.
    """

    # convert to Dublin Julian Day as used by PyEphem
    djd = jd_to_djd(jd)

    # get postion of Moon
    Moon = ephem.Moon(djd)

    # Angles in decimal degrees for ra and dec
    ra = Moon.ra * rad_to_deg
    dec = Moon.dec * rad_to_deg

    return ra, dec

In [10]:
def source_obsrange(ra, dec, zmin, zmax, azmin=0, azmax=360, t_resol=5.):
    """ Calculate the range of lst for which a given sky position is in the 
    range zmin < za < zmax.

    The result is an list with pair of values with the extremes of the 
    intervals.

    t_resol is the time resolution in minutes. Its default value is 5.
    """
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Create and array of sample lsts to calculate position
    # resolution of 5 minutes
    n_lst = math.ceil(24. * 60. / t_resol)
    lsts = numpy.linspace(0, 24, n_lst)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Calculate the position of the source on that sample lsts
    ZA = [radec_zaaz(ra, dec, lst)[0] for lst in lsts]
    AZ = [radec_zaaz(ra, dec, lst)[1] for lst in lsts]

    #~~~~~~~~~~~~~~~~~~~~~~~
    # Generate a vector with:
    # 0: Source is not observable for that lst
    # 1: Source is observable
    observable = []
    for i in range(len(ZA)):
        za = ZA[i]
        az = AZ[i]

        if (zmin <= za and za <= zmax) and (azmin <= az and az <= azmax):
            observable.append(1)
        else:
            observable.append(0)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Generate observability ranges from the observable vector  
    obs_range = obsrange_from_observable(observable, lsts)

    # plot the results for inspection
    #print obs_range
    #pylab.plot(lsts, za)
    #pylab.plot(lsts, numpy.array(observable) * 60.)
    #pylab.show()
    
    return obs_range

In [11]:
def obsrange_from_observable(observable, lsts):
    """ Take the observable and lsts vectors and returns the obs_range

    observable is a list with 0/1 that represents the observability for a
    given sky position (ra/dec) at the corresponding lst in vector lsts
    """

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Generate observability ranges from the observable vector 
    obs_range = []
    i = 0
    N = len(lsts)
    while i < N:
        
        # Find first lst of observable range
        if observable[i] == 1:
            # lower limit for range
            lst_l = lsts[i]
            
            # default upper limit, in case it is last element
            lst_u = lsts[i]

            i += 1
            while i < N:
            
                # Find first zero element
                if observable[i] == 0:
                    
                    # upper limit is previous element
                    lst_u = lsts[i - 1]

                    # save observability range
                    obs_range.append([lst_l, lst_u])
                    break

                # If last element on array close obs_range
                elif i == N - 1:
                    
                    # upper limit
                    lst_u = lsts[i]
                    
                    # save observability range
                    obs_range.append([lst_l, lst_u])

                # try next element
                i += 1

        # try next element
        i += 1

    return obs_range

In [12]:
def check_observability(obs_range, lst_obs):
    """ Check if a given lst is contained on obs_range.
    """

    # wrap around to [0, 24) range
    lst_obs = lst_obs % 24

    # default observability value
    observable = False

    # Loop through the observing ranges and check is lst_obs is in at least one
    for range in obs_range:

        if range[0] <= lst_obs and lst_obs <= range[1]:
            observable = True
            break

    return observable

In [13]:
def closest_obsrange(obs_range, lst_obs):
    """ Determine the closest observing range

    This is used to determine the observing range that goes into schedules
    start and stop ranges.

    Given an observing range tries:
    
    1) See if lst_obs is inside one of the ranges in obs_range and return that
    
    2) If 1) doesn't work, finds the range that is inmediately after
    returns it

    NOTE: Be careful about ranges that finish in 24.00, those might continue on
    0.0 and went for some extra time

    A simmilar situation for the case when the range starts at 0.0.
    """

    # wrap around to [0, 24) range
    lst_obs = lst_obs % 24

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    def extend_range(obs_range, range):
        """ Internal function
        
        This function extends a range if there is a contiguous one
        """
        
        # lower and upper range limits
        range_l = range[0]
        range_u = range[1]

        # check if window continues at 00 but doesn't start at 00
        if range_u == 24.0 and range_l != 0.0:
                
            # loop through ranges
            for alt_range in obs_range:
                    
                # if range starting at 00, join with previous range
                if alt_range[0] == 0.0:
                        
                    range_u = alt_range[1]
            
        # check if window continues before 00 but doesn't end at 24.0
        elif range_l == 0.0 and range_u != 24.0:
                
            # loop through ranges
            for alt_range in obs_range:
                    
                # if range finishing at 24.0, join with previous range
                if alt_range[1] == 24.0:

                    range_l = alt_range[0]

        return range_l, range_u


    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # try to find range that contains lst_obs
    for range in obs_range:
        
        # lst in range
        if range[0] <= lst_obs and lst_obs <= range[1]:
            
            # extend the range if necessary
            range_l, range_u = extend_range(obs_range, range)

            # save range
            closest_range = [range_l, range_u]
            
            return closest_range

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # if lst_obs not inside a given range find next range
    for range in obs_range:

        # this is the next range
        if lst_obs <= range[0]:

            # extend the range if necessary
            range_l, range_u = extend_range(obs_range, range)

            # save range
            closest_range = [range_l, range_u]
            
            return closest_range
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # if lst_obs doesn't have next range it could have previous
    for range in obs_range:

        # this is the previous range
        if range[1] <= lst_obs:

            # extend the range if necessary
            range_l, range_u = extend_range(obs_range, range)

            # save range
            closest_range = [range_l, range_u]
            
            return closest_range
       
        
    # default return value
    return []

In [14]:
def wait_time_for_observability(obs_range, lst_obs):
    """ Return the time to wait before source is on observability range.

    d_lst_wait:     Wait time returned is in decimal hours
    """

    # wrap around for [0, 24) range
    lst_obs = lst_obs % 24

    # check if source is observable, if so return 0
    if check_observability(obs_range, lst_obs):
        d_lst_wait = 0.0
        
    # find the wait time
    else:
        
        # start time is first element of closest obsrange
        lst_start = closest_obsrange(obs_range, lst_obs)[0]

        # two cases for wait time, be careful
        if lst_obs <= lst_start:
            d_lst_wait = lst_start - lst_obs

        else:
            d_lst_wait = (24.0 - lst_obs) + lst_start
        
    return d_lst_wait

In [15]:
def intersect_obs_ranges(obs_ranges, t_resol=0.01):
    """ Given a list of observing ranges, calculate their intersection

    t_resol is the time resolution in minutes. Its default value is 0.01 minute
    """

    # make lsts vector with given time resolution
    n_lst = math.ceil(24. * 60. / t_resol)
    lsts = numpy.linspace(0, 24, n_lst)

    # calculate observable vectors for intersection
    observable = []
    for lst in lsts:
        
        # initialize observability flag, observable unlees opposite proved
        observable_at_lst = True
        
        # goes through the list of observing ranges
        for obs_range in obs_ranges:
            
            # check observability for particular range
            if check_observability(obs_range, lst) == False:
                observable_at_lst = False      

        # Test abservability for all regions
        if observable_at_lst:
            observable.append(1)
        else:
            observable.append(0)
                

    # get the combined obs_range
    obs_range = obsrange_from_observable(observable, lsts)

    return obs_range

In [16]:
def lst_range_to_obs_range(lst_start, lst_stop):
    """ Given an lst_start an lst_stop returns the equivalent obs_range
    """

    # convert to [0, 24) range
    lst_start = lst_start % 24.0
    lst_stop = lst_stop % 24.0

    # two cases for obs_range
    
    if lst_start <= lst_stop:
        
        obs_range = [[lst_start, lst_stop]]

    else:
        
        obs_range = [[0.0, lst_stop], [lst_start, 24.0]]

    
    return obs_range

In [17]:
def dec_to_sex(dec, out_type='list', dec_plac=0, floor=True, time=True):
    """ Convert a decimal number to sexigesimal format

    out_type is the type of output required
    
    out_type = 'list',    [dd, mm, ss]
    out_type = 'string'   'dd:mm:ss'

    dec_plac:  Number of decimal places for seconds
    floor:  Take the floor for seconds
    time:   0-23 range for hh
    """

    # determine sign of the number
    if dec < 0:
        sign = -1
        dec = -dec
    else:
        sign = 1

    # calculate components
    dd = math.floor(dec)
    mm = math.floor( (dec - dd) * 60.0 )
    ss = (dec - dd - mm / 60.0) * 3600.0

    # Take the floor for seconds
    if floor:
        ss = math.floor(ss)

    # Use 0-23 range for time
    if time:
        dd = dd % 24

    # Check the output type
    if out_type == 'string':

        # format depend on number of decimal places
        if dec_plac == 0:
            format = '%02d:%02d:%02.0f'
        else:
            format =\
                '%02d:%02d:%0' + str(dec_plac + 3) + '.' + str(dec_plac) + 'f'

        sex = format % (dd, mm, ss)
        
        # add sign if necessary
        if sign == -1:
            sex = '-' + sex

    elif out_type == 'list':
        sex = [dd, mm, ss]

        # add sign if necessary
        if sign == -1:
            if dd != 0:
                sex[0] = -sex[0]
            elif mm != 0:
                sex[1] = -sex[1]
            else:
                sex[2] = -sex[2]

    return sex

In [18]:
def sex_to_dec(sex):
    """ Convert a sexigesimal number to decimal

    To make the conversion consider that the zero goes on the first nonzero
    component of the sexigesimal representation of the number.
    """

    # get the negative sign if any
    if sex[0] < 0 or sex[1] < 0 or sex[2] < 0:
        sign = -1.0
    else:
        sign = 1.0

    dec = sign * (abs(sex[0]) + abs(sex[1]) / 60.0 + abs(sex[2]) / 3600.0)

    return dec

In [19]:
def parallactic_angle_lst(ra, dec, lst, lat=tel_data.lat):
    """Compute the parallactic angle given the LST and latitude.                                                                                                        
                                                                                                                                                                        
    Sign convention is that parallactic angle > 0 when an object is                                                                                                       
    west of the meridian.                                                                                                                                                 
                                                                                                                                                                          
    Ra and Dec in degrees, lst in hrs,  Returns degrees.                                                                                                                
                                                                                                                                                                       
    """
    #first need to convert everything to radians
    ra_rad = ra*deg_to_rad
    dec_rad = dec*deg_to_rad

    #get lst to right range
    while(lst >= 24):
        lst -= 24
    lst_rad = lst*15*deg_to_rad
    lat_rad = lat*deg_to_rad
    ha=lst_rad-ra_rad
    pa = numpy.arctan2(numpy.cos(lat_rad)*numpy.sin(ha),
                   numpy.cos(dec_rad)*numpy.sin(lat_rad)-numpy.sin(dec_rad)*numpy.cos(lat_rad)*numpy.cos(ha))

    #then convert back to degrees for returning
    pa_deg = pa*rad_to_deg
    return pa_deg